In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
data_folder = '/content/drive/MyDrive/235kaggle'
import os
os.listdir(data_folder)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

def read_csv(file_name, parse_dates=None):
    return pd.read_csv(os.path.join(data_folder, file_name), parse_dates=parse_dates)

logon = read_csv('logon.csv', parse_dates=["date"])
devices = read_csv('device.csv', parse_dates=["date"])
emails = read_csv('email.csv', parse_dates=["date"])
files = read_csv('file.csv', parse_dates=["date"])

print("Logon shape:", logon.shape)
print("Devices shape:", devices.shape)
print("Emails shape:", emails.shape)
print("Files shape:", files.shape)

display(logon.head())
display(devices.head())
display(emails.head())
display(files.head())

In [ ]:
# date ranges
print("\nLogon date range:", logon["date"].min(), "->", logon["date"].max())
print("Device date range:", devices["date"].min(), "->", devices["date"].max())

# logon events by hour of day
logon["hour"] = logon["date"].dt.hour

# activity distribution
plt.figure(figsize=(8, 4))
logon["hour"].hist(bins=24)
plt.xlabel("Hour of day")
plt.ylabel("Number of logon/logoff events")
plt.title("Distribution of logon/logoff events by hour")
plt.tight_layout()
plt.show()


In [ ]:
logon_df = logon.copy()
devices_df = devices.copy()
emails_df = emails.copy()
files_df = files.copy()


logon_df = logon_df.rename(columns={"date": "timestamp"})
devices_df = devices_df.rename(columns={"date": "timestamp"})
emails_df = emails_df.rename(columns={"date": "timestamp"})
files_df = files_df.rename(columns={"date": "timestamp"})

for df in [logon_df, devices_df, emails_df, files_df]:
    df["date_only"] = df["timestamp"].dt.date
    df["hour"] = df["timestamp"].dt.hour
    df["dayofweek"] = df["timestamp"].dt.dayofweek

# helper: count night/odd-hour events
def count_late_hours(hours: pd.Series, start=0, end=6, night_start=20) -> int:
    """
    Count events between [0,6) or [20,24).
    """
    return ((hours < end) | (hours >= night_start)).sum()

logon_daily = (
    logon_df.groupby(["user", "date_only"])
    .agg(
        total_logon_events=("activity", "count"),
        unique_logon_pcs=("pc", pd.Series.nunique),
        night_logon_events=("hour", count_late_hours),
        min_logon_time=("timestamp", "min"),
        max_logon_time=("timestamp", "max"),
    )
    .reset_index()
)

logon_daily["work_span_hours"] = (
    (logon_daily["max_logon_time"] - logon_daily["min_logon_time"]).dt.total_seconds()
    / 3600.0
)

devices_daily = (
    devices_df.groupby(["user", "date_only"])
    .agg(
        total_device_events=("activity", "count"),
        device_connects=("activity", lambda s: (s == "Connect").sum()),
        unique_device_pcs=("pc", pd.Series.nunique),
        night_device_events=("hour", count_late_hours),
    )
    .reset_index()
)

emails_daily = (
    emails_df.groupby(["user", "date_only"])
    .agg(
        total_email_events=("timestamp", "count"),
        unique_email_pcs=("pc", pd.Series.nunique),
        night_email_events=("hour", count_late_hours),
    )
    .reset_index()
)

email_recip = emails_df.copy()
email_recip["recip_blob"] = (
    email_recip["to"].fillna("").astype(str) + ";" +
    email_recip["cc"].fillna("").astype(str) + ";" +
    email_recip["bcc"].fillna("").astype(str)
)
email_recip["recipient"] = email_recip["recip_blob"].str.replace(",", ";").str.split(";")
email_recip = email_recip.explode("recipient")
email_recip["recipient"] = email_recip["recipient"].astype(str).str.strip()
email_recip = email_recip[email_recip["recipient"] != ""]

email_recip_daily = (
    email_recip.groupby(["user", "date_only"])
    .agg(unique_email_recipients=("recipient", pd.Series.nunique))
    .reset_index()
)

emails_daily = emails_daily.merge(email_recip_daily, on=["user", "date_only"], how="left")

files_df["is_copy"] = files_df["activity"].astype(str).str.contains("copy", case=False, na=False)

files_daily = (
    files_df.groupby(["user", "date_only"])
    .agg(
        total_file_events=("activity", "count"),
        unique_file_pcs=("pc", pd.Series.nunique),
        unique_files_touched=("filename", pd.Series.nunique),
        file_copy_events=("is_copy", "sum"),
    )
    .reset_index()
)

files_daily = files_daily[["user", "date_only", "total_file_events", "unique_files_touched", "file_copy_events"]]

daily = logon_daily.merge(
    devices_daily, on=["user", "date_only"], how="outer"
).merge(
    emails_daily, on=["user", "date_only"], how="outer"
).merge(
    files_daily, on=["user", "date_only"], how="outer"
)

numeric_cols = daily.select_dtypes(include=[np.number]).columns
daily[numeric_cols] = daily[numeric_cols].fillna(0)

print("Daily user-day table shape:", daily.shape)
display(daily.head())


In [ ]:
# feature columns for anomaly detection
feature_cols = [
    "total_logon_events",
    "unique_logon_pcs",
    "night_logon_events",
    "work_span_hours",

    "total_device_events",
    "device_connects",
    "unique_device_pcs",
    "night_device_events",

    "total_email_events",
    "unique_email_pcs",
    "night_email_events",
    "unique_email_recipients",

    "total_file_events",
    "unique_files_touched",
    "file_copy_events",
]

print("Using feature columns:", feature_cols)

X = daily[feature_cols].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

iso = IsolationForest(
    n_estimators=200,
    contamination=0.02,
    random_state=42,
    n_jobs=-1,
)

iso.fit(X_scaled)

daily["anomaly_score"] = iso.decision_function(X_scaled)
daily["anomaly_label"] = iso.predict(X_scaled)

num_anomalies = (daily["anomaly_label"] == -1).sum()
print("Number of anomalous user-days:", num_anomalies, "out of", len(daily))

display(daily.head())
print("daily.shape:", daily.shape)


In [ ]:
# sort anomalies by most anomalous (lowest score)
anomalies = (
    daily[daily["anomaly_label"] == -1]
    .sort_values("anomaly_score")
    .reset_index(drop=True)
)

print("Top 20 anomalies (user-day rows):")
display(anomalies[["user", "date_only", "anomaly_score"] + feature_cols].head(20))

plt.figure(figsize=(8, 4))
daily["anomaly_score"].hist(bins=50)
plt.xlabel("IsolationForest anomaly score")
plt.ylabel("Count of user-days")
plt.title("Distribution of anomaly scores")
plt.tight_layout()
plt.show()

if len(anomalies) > 0:
    sus_user = anomalies["user"].iloc[0]
    print("Inspecting user:", sus_user)

    sus_user_days = daily[daily["user"] == sus_user].sort_values("date_only")

    display(
        sus_user_days[
            ["user", "date_only", "anomaly_score", "anomaly_label"] + feature_cols
        ].tail(30)
    )

    plt.figure(figsize=(10, 4))
    plt.plot(sus_user_days["date_only"], sus_user_days["anomaly_score"], marker="o")
    plt.xticks(rotation=45)
    plt.xlabel("Date")
    plt.ylabel("Anomaly score")
    plt.title(f"Anomaly scores over time for user {sus_user}")
    plt.tight_layout()
    plt.show()
else:
    print("No anomalies detected with current contamination setting.")


In [ ]:
search_user = "CDE1846"

user_data = daily[daily["user"].str.contains(search_user, case=False, na=False)]

if len(user_data) == 0:
    print(f"No users found containing '{search_user}'.")
else:
    print(f"Showing records for user(s) matching: {search_user}")
    display(
        user_data.sort_values("anomaly_score")[
            ["user", "date_only", "anomaly_score", "anomaly_label"] + feature_cols
        ].head(30)
    )

    # plot anomaly history for each matching user
    for u in user_data['user'].unique():
        tmp = user_data[user_data["user"] == u].sort_values("date_only")
        plt.figure(figsize=(10,4))
        plt.plot(tmp["date_only"], tmp["anomaly_score"], marker="o")
        plt.xticks(rotation=45)
        plt.xlabel("Date")
        plt.ylabel("Anomaly Score")
        plt.title(f"Anomaly scores over time — {u}")
        plt.tight_layout()
        plt.show()

In [ ]:
daily["anomaly_rank"] = daily["anomaly_score"].rank(method="min", ascending=True).astype(int)

n = len(daily)
daily["anomaly_percentile"] = (daily["anomaly_rank"] - 1) / (n - 1) if n > 1 else 0.0
daily["anomaly_percentile_100"] = 100.0 * daily["anomaly_percentile"]

# date range filter
start_date = "2011-04-21"
end_date   = "2011-04-25"

start_date = pd.to_datetime(start_date).date()
end_date   = pd.to_datetime(end_date).date()

daily_range = daily[(daily["date_only"] >= start_date) & (daily["date_only"] <= end_date)].copy()

search_user = "CDE1846"
if search_user is not None:
    daily_range = daily_range[daily_range["user"].str.contains(search_user, case=False, na=False)].copy()
    print("Rows after user filter:", len(daily_range))

# display the most anomalous rows in that date range
display_cols = ["user", "date_only", "anomaly_score", "anomaly_label",
                "anomaly_rank", "anomaly_percentile_100"] + feature_cols

display(daily_range.sort_values(["anomaly_percentile_100", "anomaly_score"])[display_cols].head(50))

In [ ]:
daily["anomaly_rank"] = daily["anomaly_score"].rank(method="min", ascending=True).astype(int)

n = len(daily)
daily["anomaly_percentile"] = (daily["anomaly_rank"] - 1) / (n - 1) if n > 1 else 0.0
daily["anomaly_percentile_100"] = 100.0 * daily["anomaly_percentile"]

# date range filter
start_date = "2010-08-12"
end_date   = "2010-08-12"

start_date = pd.to_datetime(start_date).date()
end_date   = pd.to_datetime(end_date).date()

daily_range = daily[(daily["date_only"] >= start_date) & (daily["date_only"] <= end_date)].copy()

search_user = "PLJ1771"
if search_user is not None:
    daily_range = daily_range[daily_range["user"].str.contains(search_user, case=False, na=False)].copy()
    print("Rows after user filter:", len(daily_range))

# display the most anomalous rows in the date range
display_cols = ["user", "date_only", "anomaly_score", "anomaly_label",
                "anomaly_rank", "anomaly_percentile_100"] + feature_cols

display(daily_range.sort_values(["anomaly_percentile_100", "anomaly_score"])[display_cols].head(50))